<a href="https://colab.research.google.com/github/onlygoodman/dacon_shopping_reviewscore_predict/blob/main/project_shop_009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

001 - https://wikidocs.net/94600 네이버 쇼핑 리뷰 데이터 분석을 참조하여 0,1 긍부정 이진 분류를 통해 확률을 0~5로 확장하여 예측


002 - 001 코드에서 target 그대로 활용, 다중분류로 예측 시도

003 - 002 코드에 dense layer 64, 32, 16, 8 사이에 각각 dropdout 0.2~0.5 적용하여 추가학습 epoch 25로 변경 early stop 인내값 8로 변경

004 - 003에서 epoch, batch-size 등 hyper parameter search에 유의미한 결과를 얻지 못하여 hyperband를 활용하여 하이퍼파라미터 튜닝 시도

005 - 004에서 얻은 하이퍼파라미터 튜닝 결과 대입하여 val_acc 측정 시도 **최고점수

006 - 005 모델에서 배치사이즈 16, denselayer 16짜리 하나 추가

007 - kobert 활용하여 결과도출

008 - kobert의 이진분류 정확도가 높은 사실을 활용하여 긍/부정 분류 결과를 라벨링 함수에 반영

009 - 이진분류(긍부정) -> 점수 이진분류 (부정:1,2 긍정:4,5) 총 3개의 모델을 통해 결과도출

010 - 009 결과 개선을 위해 007모델을 사용하여 class 별로 분포를 고려하여 가중치 부여 후 학습

# pip 및 kobert- colab환경에서 구축

In [1]:
!pip install gluonnlp pandas tqdm   
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install torch

!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

import os
from zipfile import ZipFile
import torch
from transformers import BertModel
import gluonnlp as nlp

from kobert import download, get_tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 344 kB 5.0 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595731 sha256=b3f56aeb45518493661e9225a963093c0997c2e83a546ce449999b839bac6fba
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49.1 MB 1.4 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg

In [2]:
def get_pytorch_kobert_model(ctx="cpu", cachedir=".cache"):
    def get_kobert_model(model_path, vocab_file, ctx="cpu"):
        bertmodel = BertModel.from_pretrained(model_path, return_dict=False)
        device = torch.device(ctx)
        bertmodel.to(device)
        bertmodel.eval()
        vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(
            vocab_file, padding_token="[PAD]"
        )
        return bertmodel, vocab_b_obj

    pytorch_kobert = {
        "url": "s3://skt-lsl-nlp-model/KoBERT/models/kobert_v1.zip",
        "chksum": "411b242919",  # 411b2429199bc04558576acdcac6d498
    }

    # download model
    model_info = pytorch_kobert
    model_path, is_cached = download(
        model_info["url"], model_info["chksum"], cachedir=cachedir
    )
    cachedir_full = os.path.expanduser(cachedir)
    zipf = ZipFile(os.path.expanduser(model_path))
    zipf.extractall(path=cachedir_full)
    model_path = os.path.join(os.path.expanduser(cachedir), "kobert_from_pretrained")
    # download vocab
    vocab_path = get_tokenizer()
    return get_kobert_model(model_path, vocab_path, ctx)


if __name__ == "__main__":
    import torch
    from kobert import get_pytorch_kobert_model

    input_ids = torch.LongTensor([[31, 51, 99], [15, 5, 0]])
    input_mask = torch.LongTensor([[1, 1, 1], [1, 1, 0]])
    token_type_ids = torch.LongTensor([[0, 0, 1], [0, 1, 0]])
    model, vocab = get_pytorch_kobert_model()
    sequence_output, pooled_output = model(input_ids, input_mask, token_type_ids)
    print(pooled_output.shape)
    print(vocab)
    print(sequence_output[0])

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]
torch.Size([2, 768])
Vocab(size=8002, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
tensor([[-0.2461,  0.2428,  0.2590,  ..., -0.4861, -0.0731,  0.0756],
        [-0.2478,  0.2420,  0.2552,  ..., -0.4877, -0.0727,  0.0754],
        [-0.2472,  0.2420,  0.2561,  ..., -0.4874, -0.0733,  0.0765]],
       grad_fn=<SelectBackward0>)


In [3]:
from kobert_tokenizer import KoBERTTokenizer
from kobert import get_pytorch_kobert_model
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab  = get_pytorch_kobert_model()

Downloading:   0%|          | 0.00/363k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/432 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

#GPU 사용 시
device = torch.device("cuda:0")

# dataset 준비

In [5]:
# Train dataset은 Label이 존재하지 않음
train_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit = pd.read_csv("/content/drive/MyDrive/shopping_dataset/sample_submission.csv")

In [6]:
train_data

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
4,4,튼튼하고 손목을 잘 받쳐주네요~,5
...,...,...,...
24995,24995,일단향이너무너무좋은데지속력만좋다면좋겠지만..워낙저렴해그래도좋아여,5
24996,24996,동일한 업체에서 발송하고 한 상자로 배송되었는데 택배비는 이중 처리되었습니다.,2
24997,24997,향이 좋아요!! 다른 향도 시켜보고 싶어요,4
24998,24998,펼쳐보니 끝부분쯤에 50cm정도 찟어져 있습니다 날짜관계로 구냥 사용합니다,2


In [7]:
train_negative = train_data[(train_data['target']==1)|(train_data['target']==2)]
train_positive = train_data[(train_data['target']==4)|(train_data['target']==5)]

In [8]:
train_negative.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,2
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,1
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,2
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,2
5,5,배송은 먼저 구입하신 분들께서 오래걸린다고 하여 참고 기달렸습니다 생각보다 일찍도착...,2


In [9]:
train_positive.head()

,id,reviews,target
4,4,튼튼하고 손목을 잘 받쳐주네요~,5
6,6,타이밍이 좋았는지 배송도 빠르고 맛있었어요! 크기가 두가지가 있길래 큰걸 사봤는데 ...,5
8,8,재구매 평소 사용하던 제품이에요. 한번에 쟁여두려 여러개 삽니다 .,5
10,10,재구매 항상 여기서만 주문해서 잘 쓰고있어요 후딱쓰기좋아요 크기도 좋구,5
12,12,재구매 좋은제품 저렴하게 잘 구매했습니다.,5


In [10]:
def changetarget(data): #긍부정 라벨링
  if data ==1:
    return 0
  elif data ==2:
    return 1

  elif data ==4:
    return 2
  elif data ==5:
    return 3

def changeposit(data): #긍정 라벨링
  if data ==4:
    return 0
  
  elif data ==5:
    return 1


def changenega(data): #부정 라벨링
  if data ==1:
    return 0
  
  elif data ==2:
    return 1

train_data['target']=train_data['target'].apply(lambda x : changetarget(x))
train_positive['target']=train_positive['target'].apply(lambda x : changeposit(x))
train_negative['target']=train_negative['target'].apply(lambda x : changenega(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [11]:
#라벨링확인
for i in [0,1]: #12500 12500
  print(train_data[train_data['target']==i]['target'].count())

for i in [0,1]: #sum = 12500
  print(train_positive[train_positive['target']==i]['target'].count())

for i in [0,1]: #sum = 12500
  print(train_negative[train_negative['target']==i]['target'].count())

4500
8000
2500
10000
4500
8000


In [12]:
#train_data에 대해 긍부정 라벨링 data list에 담음
data_all_list = []
for ques, label in zip(train_data['reviews'], train_data['target'])  :
    data = []   
    data.append(ques)
    data.append(str(label))

    data_all_list.append(data)

# #train_positive 대해 긍부정 라벨링 data list에 담음
# data_pos_list = []
# for ques, label in zip(train_positive['reviews'], train_positive['target'])  :
#     data = []   
#     data.append(ques)
#     data.append(str(label))

#     data_pos_list.append(data)

# #train_negative에 대해 긍부정 라벨링 data list에 담음
# data_neg_list = []
# for ques, label in zip(train_negative['reviews'], train_negative['target'])  :
#     data = []   
#     data.append(ques)
#     data.append(str(label))

#     data_neg_list.append(data)

In [13]:
print(data_all_list[:3])
# print(data_pos_list[:3])
# print(data_neg_list[:3])

[['조아요 처음구입 싸게햇어요', '1'], ['생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 아깝네요', '0'], ['디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져있네요 ㅠ 귀찮아서 그냥쓰려구요 배송기사님은 친절하셨어요', '1']]


In [14]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         

    def __len__(self):
        return (len(self.labels))

# tokenizer install 및 절차확인

In [15]:
!pip install --no-cache-dir transformers sentencepiece 

from transformers import AutoTokenizer, AutoModelForMaskedLM

# kcbert의 tokenizer와 모델을 불러옴.
kcbert_tokenizer = AutoTokenizer.from_pretrained("beomi/kcbert-base")
kcbert = AutoModelForMaskedLM.from_pretrained("beomi/kcbert-base")

result = kcbert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
print(kcbert_tokenizer.vocab['대선'])
print([kcbert_tokenizer.encode(token) for token in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/244k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


['너는', '내년', '대선', '때', '투표', '##할', '수', '있어', '?']
9311
[[2, 9039, 3], [2, 8946, 3], [2, 9311, 3], [2, 1003, 3], [2, 8701, 3], [2, 7, 7, 3358, 3], [2, 1931, 3], [2, 8749, 3], [2, 32, 3]]


In [16]:
!pip install --no-cache-dir transformers sentencepiece

from transformers import AutoModel, AutoTokenizer

kobert_tokenizer = AutoTokenizer.from_pretrained("skt/kobert-base-v1", use_fast=False)
kobert = AutoModel.from_pretrained("skt/kobert-base-v1")

result = kobert_tokenizer.tokenize("너는 내년 대선 때 투표할 수 있어?")
print(result)
kobert_vocab = kobert_tokenizer.get_vocab()
print(kobert_vocab.get('▁대선'))
print([kobert_tokenizer.encode(token) for token in result])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Downloading:   0%|          | 0.00/535 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/352M [00:00<?, ?B/s]

['▁너', '는', '▁내년', '▁대선', '▁때', '▁투표', '할', '▁수', '▁있어', '?']
1654
[[1457, 3, 2], [517, 5760, 3, 2], [1437, 3, 2], [1654, 3, 2], [1844, 3, 2], [4772, 3, 2], [4977, 3, 2], [2872, 3, 2], [3868, 3, 2], [633, 3, 2]]


# kobert model 학습단계
model1

model2

model3

1.   try1
- 성능
    - model1 acc 0.9287974683544303 P or N
    - model2 acc 0.7796875 P label
    - model3 acc 0.63671875 N label

- parameters (model 1~3)

    - max_len = 64
    - batch_size = 64
    - warmup_ratio = 0.1
    - num_epochs = 5  
    - max_grad_norm = 1
    - log_interval = 200
    - learning_rate =  5e-5 



#model 
1, 2, 4, 5 예측을 위해 손실함수의 클래스별 가중치 부여

문장의 점수 라벨링을 해주는 모델

return 0 1 2 3
answer 1 2 4 5

In [17]:
#model 1

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [18]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_all_list, test_size=0.2, shuffle=True, random_state=42)

In [19]:
tok=tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

In [20]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [21]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=4,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
#BERT 모델 불러오기
model1 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model1.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model1.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
w =torch.cuda.FloatTensor([0.3, 0.2, 0.273, 0.227])  # torch.tensor() 
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(weight = w) # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [30]:
w.shape

torch.Size([4])

In [31]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model1.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model1.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model1.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model1(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/313 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 1.364076852798462 train acc 0.34375
epoch 1 batch id 201 loss 0.642960250377655 train acc 0.6893656716417911
epoch 1 train acc 0.7044728434504792


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.6366693037974683


  0%|          | 0/313 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.30447059869766235 train acc 0.875
epoch 5 batch id 201 loss 0.19555220007896423 train acc 0.9119247512437811
epoch 5 train acc 0.9183306709265175


  0%|          | 0/79 [00:00<?, ?it/s]

epoch 5 test acc 0.6503164556962026


In [32]:
#라벨링
def predict1(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0, pin_memory=True)
    
    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(1)
            
            elif np.argmax(logits) == 1:
                test_eval.append(2)
            
            elif np.argmax(logits) == 2:
                test_eval.append(4)
            
            elif np.argmax(logits) == 3:
                test_eval.append(5)
            
        # print(">> 입력하신 내용의 평점은 {}점 입니다.".format(test_eval[0]))
        return test_eval[0]

In [33]:
test_sen = test_data.reviews

pred1 = []

for sentence in tqdm_notebook(test_sen):
  pred1.append(predict1(sentence))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25000 [00:00<?, ?it/s]

In [34]:
projectname = "shop"
model = "KOBERT_add_weight"
testnum = "010"

submit1 = submit.copy()
submit1["target"] = pred1

# test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit1.to_csv("/content/drive/MyDrive/shopping_dataset/sub_{}_{}_model_{}.csv".format(testnum, projectname, model),index=False)

In [35]:
submit1.head()

,id,target
0,0,2
1,1,1
2,2,5
3,3,1
4,4,1


In [36]:
sentest= ["아진짜 좋아", "이거 대박", "정말 좋긴한데.. 뭔가 아쉬워요", "이거 진짜 진짜 너무 너무 별로다 ... 조상님이 울고가겠어요"]

for sen1 in sentest:
  print(sen1)
  print(predict1(sen1))
  print()

아진짜 좋아
2

이거 대박
5

정말 좋긴한데.. 뭔가 아쉬워요
2

이거 진짜 진짜 너무 너무 별로다 ... 조상님이 울고가겠어요
1



#model2

긍정으로 분류된 문장을

4점과 5점으로 분류해주는 model

return 0 == 4

return 1 == 5

In [83]:
#model 2

# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [49]:
dataset_train, dataset_test = train_test_split(data_pos_list, test_size=0.2, shuffle=True, random_state=42)

tok=tokenizer.tokenize
data_train2 = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test2 = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

train_dataloader2 = torch.utils.data.DataLoader(data_train2, batch_size=batch_size, num_workers=5)
test_dataloader2 = torch.utils.data.DataLoader(data_test2, batch_size=batch_size, num_workers=5)


#BERT 모델 불러오기
model2 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model2.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model2.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader2) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader2

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [50]:
train_history2=[]
test_history2=[]
loss_history2=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model2.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader2)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model2.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history2.append(train_acc / (batch_id+1))
            loss_history2.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history2.append(train_acc / (batch_id+1))
    
    model2.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader2)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model2(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history2.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/157 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.83506178855896 train acc 0.21875
epoch 1 train acc 0.7524880573248408


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.78125


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.20195113122463226 train acc 0.921875
epoch 5 train acc 0.9101313694267515


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 5 test acc 0.7796875


In [51]:
#긍정 라벨링 예측
def predict2(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader2 = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0, pin_memory=True)
    
    model2.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader2):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model2(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(4)
            elif np.argmax(logits) == 1:
                test_eval.append(5)

        # print(">> 입력하신 내용의 평점은 {}점 입니다.".format(test_eval[0]))
        return test_eval[0]

In [52]:
test_sen = test_data.reviews

pred2 = []

for sentence in tqdm_notebook(test_sen):
  pred2.append(predict2(sentence))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25000 [00:00<?, ?it/s]

In [53]:
projectname = "shop"
modelname = "KOBERT_pos"
testnum = "009"

submit2 = submit.copy()
submit2["target"] = pred2

# test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit2.to_csv("/content/drive/MyDrive/shopping_dataset/sub_{}_{}_model_{}.csv".format(testnum, projectname, modelname),index=False)

#model3

부정 라벨링 모델

return 0 = 1

return 1 = 2


In [54]:
dataset_train, dataset_test = train_test_split(data_neg_list, test_size=0.2, shuffle=True, random_state=42)

tok=tokenizer.tokenize
data_train3 = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test3 = BERTDataset(dataset_test,0, 1, tok, vocab,  max_len, True, False)

train_dataloader3 = torch.utils.data.DataLoader(data_train3, batch_size=batch_size, num_workers=5)
test_dataloader3 = torch.utils.data.DataLoader(data_test3, batch_size=batch_size, num_workers=5)


#BERT 모델 불러오기
model3 = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
 
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model3.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model3.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 대표적인 loss func

t_total = len(train_dataloader3) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader3

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [55]:
train_history3=[]
test_history3=[]
loss_history3=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model3.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader3)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model3(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model3.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history3.append(train_acc / (batch_id+1))
            loss_history3.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model3.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader3)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model3(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history3.append(test_acc / (batch_id+1))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/157 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.6875601410865784 train acc 0.578125
epoch 1 train acc 0.616640127388535


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

epoch 4 test acc 0.643359375


  0%|          | 0/157 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.2997105121612549 train acc 0.890625
epoch 5 train acc 0.8900278662420382


  0%|          | 0/40 [00:00<?, ?it/s]

epoch 5 test acc 0.63671875


In [56]:
#부정 라벨링 예측
def predict3(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader3 = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=0, pin_memory=True)
    
    model3.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader3):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model3(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(1)
            elif np.argmax(logits) == 1:
                test_eval.append(2)

        # print(">> 입력하신 내용의 평점은 {}점 입니다.".format(test_eval[0]))
        return test_eval[0]

In [57]:
test_sen = test_data.reviews

pred3 = []

for sentence in tqdm_notebook(test_sen):
  pred3.append(predict3(sentence))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/25000 [00:00<?, ?it/s]

In [59]:
projectname = "shop"
modelname = "KOBERT_neg"
testnum = "009"

submit3 = submit.copy()
submit3["target"] = pred3

# test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit3.to_csv("/content/drive/MyDrive/shopping_dataset/sub_{}_{}_model_{}.csv".format(testnum, projectname, modelname),index=False)

# model 통합하여 예측

In [62]:
#model1, model2, model3 한번에 사용한 예측

test_sen = test_data.reviews

pred_final = []

for sentence in tqdm_notebook(test_sen):
  result1 = predict1(sentence)

  if result1 == 0 : #부정이면 부정모델 라벨링 예측하여 추가
    pred_final.append(predict3(sentence))

  elif result1 == 1: #긍정이면 긍정모델 라벨링 예측하여 추가
    pred_final.append(predict2(sentence))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/25000 [00:00<?, ?it/s]

In [64]:
projectname = "shop"
modelname = "KOBERT_Final"
testnum = "009"

submit_final = submit.copy()
submit_final["target"] = pred_final

# test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit_final.to_csv("/content/drive/MyDrive/shopping_dataset/sub_{}_{}_model_{}.csv".format(testnum, projectname, modelname),index=False)

# 결과값 통합하여 예측

In [66]:
submit_final.head()

,id,target
0,0,2
1,1,5
2,2,5
3,3,5
4,4,5


In [74]:
submit1.head()

,id,target
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0


In [75]:
submit2.head()

,id,target
0,0,4
1,1,4
2,2,5
3,3,5
4,4,4


In [76]:
submit3.head()

,id,target
0,0,2
1,1,1
2,2,2
3,3,1
4,4,1


In [79]:
submit['PN']=submit1['target']
submit['PL']=submit2['target']
submit['NL']=submit3['target']

submit.head()

,id,target,PN,PL,NL
0,0,0,0,4,2
1,1,0,0,4,1
2,2,0,1,5,2
3,3,0,0,5,1
4,4,0,0,4,1


In [80]:
t = submit.PN #P or N
p = submit.PL #P labeling
n = submit.NL #N labeling

answer=[]
for a,b,c in zip(t, p, n):
  if a==0:
    answer.append(c)
  
  elif a==1:
    answer.append(b)
    
len(answer)
  

25000

In [81]:
projectname = "shop"
modelname = "KOBERT_Final"
testnum = "010"

submit_final = pd.read_csv('/content/drive/MyDrive/shopping_dataset/sample_submission.csv')
submit_final["target"] = answer

# test_data = pd.read_csv("/content/drive/MyDrive/shopping_dataset/test.csv")
submit_final.to_csv("/content/drive/MyDrive/shopping_dataset/sub_{}_{}_model_{}.csv".format(testnum, projectname, modelname),index=False)

In [82]:
submit_final.head()

,id,target
0,0,2
1,1,1
2,2,5
3,3,1
4,4,1


# validation part

In [71]:
def finalpredict(sentence):
  result1 = predict1(sentence)

  if result1 == 0 : #부정이면 부정모델 라벨링 예측하여 추가
    return(predict3(sentence))

  elif result1 == 1: #긍정이면 긍정모델 라벨링 예측하여 추가
    return(predict2(sentence))

In [72]:
senlist = ["조아용", "이건 말도 안되는 제품이야..", "조상님이 좋아하겠어요"]

for sen in senlist:
  print(sen, finalpredict(sen))

조아용 2
이건 말도 안되는 제품이야.. 5
조상님이 좋아하겠어요 2


In [70]:
train_data.head()

,id,reviews,target
0,0,조아요 처음구입 싸게햇어요,0
1,1,생각보다 잘 안돼요 매지 바른지 하루밖에 안됐는데ㅠㅠ 25천원가량 주고 사기 너무 ...,0
2,2,디자인은괜찮은데 상품이 금이가서 교환했는데 두번째받은상품도 까져있고 안쪽에 금이가져...,0
3,3,기전에 이 제품말고 이마트 트레이더스에서만 팔던 프리미엄 제품을 사용했었습니다. 샘...,0
4,4,튼튼하고 손목을 잘 받쳐주네요~,1


#Finish

In [1]:
import pandas as pd

subtoday = pd.read_csv('/content/drive/MyDrive/shopping_dataset/sub_010_shop_model_KOBERT_add_weight.csv')

for i in [1,2,4,5]:
  print(subtoday[subtoday.target == i].count())


id        4369
target    4369
dtype: int64
id        8043
target    8043
dtype: int64
id        2487
target    2487
dtype: int64
id        10101
target    10101
dtype: int64
